In [ ]:
df = pd.read_pickle("../data/pepper_data_test.pkl")
len(df)

In [ ]:
import base64
import os
from pathlib import Path
import pandas as pd
import requests
import time

API_URL = "https://api.perplexity.ai/chat/completions"
API_KEY = 

df = pd.read_pickle("../data/pepper_data_test.pkl")
df = df.head(5)

actions = list(df.columns[:9])
print(f"{actions=}")

letter_to_score = {
    'A': 1,  # very inappropriate
    'B': 2,  # inappropriate
    'C': 3,  # neutral
    'D': 4,  # appropriate
    'E': 5   # very appropriate
}


In [ ]:
def encode_image_base64(image_path: str) -> str:
    with open(image_path, "rb") as image_file:
        encoded_bytes = base64.b64encode(image_file.read())
    base64_str = encoded_bytes.decode('utf-8')
    mime_type = "image/png"
    return f"data:{mime_type};base64,{base64_str}"

def create_prompt(action: str) -> str:
    # standarise actions to an -ing form to be fitting in a prompt
    action_ing = action.replace("Carry", "Carrying") if "Carry" in action else action
    action_ing = action_ing.lower()
    return (
        f"""
        This is an image of an indoor scene with a Pepper robot present.
        Given an image of a scenario which includes a robot and several humans, a human evaluator has to answer a question about whether it is socially appropriate for the robot to carry out a certain action in the given scenario.
        Question: How socially appropriate is it for the robot to be doing the following actions [Vaccum Cleaning, Mopping the Floor, Carry Warm Food, Carry Cold Food, Carry Drinks, Carry Small Objects, Carry Large Objects, Cleaning, Starting a conversation]?
        Answer choices for each action: A. very inappropriate, B. inappropriate, C. neutral, D. appropriate, E. very appropriate.
        """
    )

def call_perplexity_api(image_data_url: str, prompt_text: str) -> str:
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "accept": "application/json"
    }
    payload = {
        "model": "sonar-pro",
        "stream": False,
        "max_tokens":18,
        "messages": [
            {"role": "system", "content": "Respond with only 9 single letters (A, B, C, D, or E) seperated by commas indicating the appropriateness level of 9 actions in the order that the actions were given, without any additional explanation or text."},
            {
                "role": "user",
                "content": [
                    {"type": "image_url", "image_url": {"url": image_data_url}},
                    {"type": "text", "text": prompt_text}
                ]
            }
        ]
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status()
    response_json = response.json()
    return response_json


In [ ]:
# Prepare output CSV path
output_csv_path = "../data/perplexity_predicted_scores.csv"
output_columns = ["image_path"] + actions

# If output CSV exists, load it to resume; otherwise prepare empty DataFrame
if os.path.exists(output_csv_path):
    output_df = pd.read_csv(output_csv_path)
    processed_images = set(output_df["image_path"].tolist())
else:
    output_df = pd.DataFrame(columns=output_columns)
    processed_images = set()

In [ ]:
for idx, row in df.iterrows():
    image_path = row['image_path']
    if image_path in processed_images:
        print(f"Skipping already processed image: {image_path}")
        continue

    if not Path(image_path).is_file():
        print(f"Image file not found: {image_path}, skipping.")
        continue
    try:
        image_data_url = encode_image_base64(str(image_path))
    except Exception as e:
        print(f"Failed to encode image {image_path}: {e}")
        continue

    predicted_scores = {}
    for action in actions:
        prompt = create_prompt(action)
        try:
            response = call_perplexity_api(image_data_url, prompt)
            model_answer = response['choices'][0]['message']['content'].strip().upper()
            if model_answer in letter_to_score:
                predicted_scores[action] = letter_to_score[model_answer]
            else:
                print(f"Unexpected API response for action '{action}' on image '{image_path}': '{response}'")
                predicted_scores[action] = None
            time.sleep(0.3)  # adjust to respect rate limits
        except Exception as e:
            print(f"API request error for action '{action}' on image '{image_path}': {e}")
            predicted_scores[action] = None
        break
    

    # Prepare record for this image
    record = {"image_path": image_path}
    for action in actions:
        record[action] = predicted_scores.get(action)

    # Append to DataFrame and save after each image
    output_df = pd.concat([output_df, pd.DataFrame([record])], ignore_index=True)
    output_df.to_csv(output_csv_path, index=False)
    print(f"Processed {idx+1}/{len(df)} images: {image_path} - saved results.")
    break

print(f"Processing complete. All results saved to {output_csv_path}")


In [ ]:
response